In [3]:
from time import timezone

import openai
import os
from typing import List
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from postgres_db.models.Article import Article

In [4]:
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))

In [1]:
from DIPLOM.code_dir.article_analysis.article_scraping.scraping import scrape_hromadske

news = scrape_hromadske()

In [5]:
news[0]

{'outlet': 'hromadske',
 'href': 'https://hromadske.ua/polityka/244063-zelenskyy-pidpysana-uhoda-pro-nadra-ye-pershym-rezultatom-zustrichi-z-trampom-u-vatykani-chekayemo-na-inshi',
 'timestamp': '2025-05-01T22:29:00+03:00',
 'title': 'Зеленський: Підписана угода про надра є першим результатом зустрічі з Трампом у Ватикані, чекаємо на інші',
 'paragraphs': ['Президент України Володимир Зеленський заявив, що підписання угоди щодо економічного партнерства з Вашингтоном стало першим результатом «ватиканської зустрічі» з президентом США Дональдом Трампом. Глава держави чекає на «інші результати» розмови.',
  'Про це Зеленський розповів у вечірньому зверненні 1 травня.',
  'Зеленський зазначив, що тепер підписану угоду спрямують на ратифікацію у Верховну Раду, зауваживши, що Україна зацікавлена в тому, щоб не було «жодних затягувань».',
  'За словами президента, угода «значно змінилась» у процесі підготовки й тепер «справді є рівноправною» та створює можливості для «доволі вагомих» інвестиці

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime

from postgres_db.models import Article, Category, FactcheckCategory, Base

Session = sessionmaker(bind=engine)
session = Session()

def create_article(news_data):
    article = Article(
        title=news_data['title'],
        href=news_data['href'],
        outlet=news_data['outlet'],
        published_at=news_data['timestamp'],
        created_at=news_data['timestamp'],
        status='Processing',
    )
    
    for i, paragraph_text in enumerate(news_data['paragraphs']):
        article.add_paragraph(paragraph_text, i+1)
    
    for i, source_href in enumerate(news_data['sources']):
        article.add_source(source_href, i+1)
    
    if 'metrics' in news_data:
        article.add_metric(news_data['metrics'])
    
    session.add(article)
    session.commit()
    
    return article

def add_category_to_article(article_id, category_name, category_description=None):
    category = session.query(Category).filter_by(name=category_name).first()
    if not category:
        category = Category(name=category_name, description=category_description or '')
        session.add(category)
        session.commit()
    
    # Get the article
    article = session.query(Article).get(article_id)
    if not article:
        raise ValueError(f"Article with ID {article_id} not found")
    
    # Create the association
    factcheck_category = FactcheckCategory(category_id=category.id, article_id=article.id)
    session.add(factcheck_category)
    session.commit()
    
    return factcheck_category

# Run the example
if __name__ == '__main__':
    # Create tables if they don't exist
    Base.metadata.create_all(engine)
    
    # Create an article
    article = create_article(news[0])
    print(f"Created article with ID: {article.id}")


Created article with ID: 1


In [12]:
import datetime
datetime.datetime.now(datetime.timezone.utc).isoformat(timespec='seconds')

'2025-05-02T07:21:07+00:00'